# portfolio

> A module to model a portfolio's performance over the available time-scale including the impact of fees.

A notebook to explore different investment portfolio ideas and compare their historic returns, inclusive of fees

In [1]:
#| default_exp portfolio

In [2]:
#| hide
from nbdev.showdoc import *
from fastcore.test import *
import pdb
from datetime import date

In [3]:
#| export
from fastcore.utils import *

In [4]:
#| export
import pendulum

# Holding
A holding is an asset that can be bought. We need to track it's value and how much of it we own over time. We therefore need financial data which can be provided by yfinance for free.

Let's import the package and have a play. Remembering that when we are using `nbdev` we have to have imports in their own cells.

In [5]:
#| export
import yfinance as yf

In [6]:
price_history = yf.Ticker('VWELX').history(period='max', # valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
                                           interval='1d', # valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
                                           actions=False)
price_history.head()

,Open,High,Low,Close,Volume
Date,,,,,
1980-01-02,0.522507,0.522507,0.522507,0.522507,0
1980-01-03,0.520185,0.520185,0.520185,0.520185,0
1980-01-04,0.523088,0.523088,0.523088,0.523088,0
1980-01-07,0.523088,0.523088,0.523088,0.523088,0
1980-01-08,0.529474,0.529474,0.529474,0.529474,0


In [7]:
price_history.index[-1]

Timestamp('2023-05-01 00:00:00')

yfinance appears to be a good option as it has price information. One concern would be what units these are but that doesn't matter as we will convert back to cash at the end.

In [8]:
show_doc(yf.Ticker.history)

---

### TickerBase.history

>      TickerBase.history (period='1mo', interval='1d', start=None, end=None,
>                          prepost=False, actions=True, auto_adjust=True,
>                          back_adjust=False, proxy=None, rounding=False,
>                          tz=None, timeout=None, **kwargs)

:Parameters:
    period : str
        Valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
        Either Use period parameter or use start and end
    interval : str
        Valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
        Intraday data cannot extend last 60 days
    start: str
        Download start date string (YYYY-MM-DD) or _datetime.
        Default is 1900-01-01
    end: str
        Download end date string (YYYY-MM-DD) or _datetime.
        Default is now
    prepost : bool
        Include Pre and Post market data in results?
        Default is False
    auto_adjust: bool
        Adjust all OHLC automatically? Default is True
    back_adjust: bool
        Back-adjusted data to mimic true historical prices
    proxy: str
        Optional. Proxy server URL scheme. Default is None
    rounding: bool
        Round values to 2 decimal places?
        Optional. Default is False = precision suggested by Yahoo!
    tz: str
        Optional timezone locale for dates.
        (default data is returned as non-localized dates)
    timeout: None or float
        If not None stops waiting for a response after given number of
        seconds. (Can also be a fraction of a second e.g. 0.01)
        Default is None.
    **kwargs: dict
        debug: bool
            Optional. If passed as False, will suppress
            error message printing to console.

## Deposits

Deposits to our fund could be naturally specified as time series of deposits. A helper function is provided to support the creation of monthly deposits. It is important that these days lie on business days so we use `pd.bdate_range` However, this could be extended to different time periods.

In [9]:
#| export
import pandas as pd
from datetime import datetime

In [10]:
#| export
to_datetime = lambda date_string: datetime.strptime(date_string,"%d/%m/%Y")
def create_monthly_deposits(start:str,        # Date of the first montly deposit.
                            end:str,          # Date of the last monthly deposit
                            deposit:float):    # Value of monthly deposit
    dti = pd.bdate_range(start=to_datetime(start),end=to_datetime(end),freq='BM')
    deposits = [deposit]*len(dti)
    return pd.Series([deposit]*len(dti), index=dti, name='deposits')

We call our function as follows.

In [11]:
monthly_deposits = create_monthly_deposits('01/01/2022','01/04/2023',10)
monthly_deposits

2022-01-31    10
2022-02-28    10
2022-03-31    10
2022-04-29    10
2022-05-31    10
2022-06-30    10
2022-07-29    10
2022-08-31    10
2022-09-30    10
2022-10-31    10
2022-11-30    10
2022-12-30    10
2023-01-31    10
2023-02-28    10
2023-03-31    10
Freq: BM, Name: deposits, dtype: int64

## Single Holding
First we create a portfolio consisting of a single holding.

In [12]:
#| export
import matplotlib.pyplot as plt
import warnings

In [13]:
#| export
class Holding:
    "A holding for fund with data available on yfinance"
    def __init__(self,
                 fund:str,                      # Name of the fund
                 ticker:str,                    # Ticker symbol for the stock
                 product_cost:float,            # Sum of all fees expressed as a percentage
                 deposits):                     # Timeseries dataframe of deposits
                 
        
        self.fund = fund
        self.history = yf.Ticker(ticker).history(period='max', # valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
                                                 interval='1d', # valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
                                                 actions=False)
        self.product_cost = product_cost
        
        # Where we've specified a timeseries where values predate the oldest time in our ticker,
        # we give a warning and then assume that the accumulated lump sum was invested at the earliest
        # possible date.
        
        #TODO: This doesn't work if there are no deposits after the fund opened
        if self.history.index[0] > deposits.index[0]:
            initial_deposit = deposits[deposits.index <= self.history.index[0]].sum()
            deposits = deposits[deposits.index >= self.history.index[0]]
            if not deposits.empty and deposits.index[0].date() == self.history.index[0]:
                deposits.iat[0] = initial_deposit
            else:
                new_row = pd.Series([initial_deposit], index=[self.history.index[0]])
                deposits = pd.concat([new_row,deposits])
                warn_msg = f"""Deposits predate initial date of {self.history.index[0]} where prices are available for {fund}. Accumulated deposit of {initial_deposit} deposited on this date.""" 
                warnings.warn(warn_msg)
        else:
             self.history = self.history.iloc[self.history.index >= deposits.index[0]]
            
        self.history['deposits']  = deposits
        self.history['deposits']  = self.history['deposits'].fillna(0)
        
        self = self.compute_value()
        
        # A holding can never have cumulative deposits less than zero.
        
    def compute_value(self):
        
        self.history['cum_deposits'] = self.history['deposits'].cumsum()
        self.history['units']        = self.history['deposits']/self.history['Close']
        self.history['cum_units']    = self.history['units'].cumsum()
        self.history['cum_value']    = self.history['cum_units']*self.history['Close']
        self.history['fees'] = ((1+self.product_cost/100)**(1/365)-1)*self.history['cum_value']
        
        return self

Let's create our first holding.

In [14]:
single_holding = Holding('Vanguard LifeStrategy 100%','0P0000TKZO.L',0.22,create_monthly_deposits('04/01/2018','01/12/2022',1000))

Our holding should have the fund name assigned.

In [15]:
assert single_holding.fund == 'Vanguard LifeStrategy 100%'

Our holding should have a history dataframe...

In [16]:
assert isinstance(single_holding.history, pd.DataFrame)

with certain columns...

In [17]:
assert all([item in single_holding.history.columns for item in ['Open','High','Low','Close','deposits','units','cum_units','cum_value','fees']])

We could try and create a condition based on length of the dataframe but as the market is close on weekends we see that this is difficult.

In [18]:
len(single_holding.history)

1324

In [19]:
(yf.Ticker('0P0000TKZO.L').history(period='max',interval='1d',actions=False).index[-1]-to_datetime('04/01/2018')).days

1940

We can instead check the early dates in the history...

In [20]:
single_holding.history.head()

,Open,High,Low,Close,Volume,deposits,cum_deposits,units,cum_units,cum_value,fees
Date,,,,,,,,,,,
2018-01-31,20638.800781,20638.800781,20638.800781,20638.800781,0,1000.0,1000.0,0.048452,0.048452,1000.000000,0.006021
2018-02-01,20603.599609,20603.599609,20603.599609,20603.599609,0,0.0,1000.0,0.000000,0.048452,998.294418,0.006011
2018-02-02,20399.000000,20399.000000,20399.000000,20399.000000,0,0.0,1000.0,0.000000,0.048452,988.381070,0.005951
2018-02-05,19985.199219,19985.199219,19985.199219,19985.199219,0,0.0,1000.0,0.000000,0.048452,968.331418,0.005830
2018-02-06,19911.099609,19911.099609,19911.099609,19911.099609,0,0.0,1000.0,0.000000,0.048452,964.741112,0.005809


and the later dates.

In [21]:
single_holding.history.tail()

,Open,High,Low,Close,Volume,deposits,cum_deposits,units,cum_units,cum_value,fees
Date,,,,,,,,,,,
2023-04-24,29858.839844,29858.839844,29858.839844,29858.839844,0,0.0,56000.0,0.0,2.325957,69450.386424,0.418147
2023-04-25,29660.339844,29660.339844,29660.339844,29660.339844,0,0.0,56000.0,0.0,2.325957,68988.683901,0.415367
2023-04-26,29469.460938,29469.460938,29469.460938,29469.460938,0,0.0,56000.0,0.0,2.325957,68544.707716,0.412694
2023-04-27,29712.140625,29712.140625,29712.140625,29712.140625,0,0.0,56000.0,0.0,2.325957,69109.170306,0.416092
2023-04-28,29729.730469,29729.730469,29729.730469,29729.730469,0,0.0,56000.0,0.0,2.325957,69150.083531,0.416338


Now, let's check we can define deposits before a fund starts.

In [22]:
initial_deposit  = create_monthly_deposits("01/01/2018","01/02/2018",250000)
lf_ruffer_diversified = Holding("LF Ruffer Diversified","0P0001MKQK.L",1.12,initial_deposit)

C:\Users\Ryan\AppData\Local\Temp\ipykernel_29804\955690272.py:31: UserWarning: Deposits predate initial date of 2021-09-01 00:00:00 where prices are available for LF Ruffer Diversified. Accumulated deposit of 250000 deposited on this date.
  warnings.warn(warn_msg)


# Portfolio Modelling
For portfolio analysis, I'd like to explore the following features:
* Modelling a porfolio consisting of multiple funds.
* Rebalancing to target allocations.
* Rebalancing at given frequency.
    
Proposed structure would be lists of fund names, tickers, allocations and ongoing charges. A deposit rate would also need to be defined. Rebalancing times could be specified as a date series.

Future functionality might include:
* Variable allocation in time.
* At present if a fund's inception date is after the first deposit then cash deposits are accumulated until the first deposit opportunity. Another possible approach would be to split the capital just between the funds that are open at a given time. Even better would be to enable concatenation of two FixedAllocationPortfolio funds... this might also be an interesting coding challenge.

In [67]:
#| export
class FixedAllocationPortfolio:
    "A collection of holdings of funds with data available on yfinance with a fixed allocation of each deposit made."
    def __init__(self,
                 fund:list,          # List of fund names
                 ticker:list,        # List of fund tickers
                 product_cost:list,  # List of fees for each fund expressed as a percentage and comprising all fees for a given fund
                 allocation:list,    # Allocation of each deposit as a fraction. Must sum to one.
                 deposits):          # Timeseries dataframe of deposits
        
        # Check fund, ticker, product_cost and allocation are all lists of equal length
        assert(all([len(input)==len(fund) for input in [ticker, product_cost, allocation]]))
        
        # Check that allocation sums to 1
        assert(sum(allocation)==1)
        
        # Record the inputs
        self.deposits = deposits
        self.allocation = allocation
        self.fund = fund
        
        # Create a holding for each fund
        self.holdings = [Holding(fund[i],ticker[i],product_cost[i],deposits*allocation[i]) for i in range(len(fund))]
        
        # Let's ensure all holdings have the same number of dates
        n_rows = [holding.history.shape[0] for holding in self.holdings]
        if len(set(n_rows)) != 1:
            max_rows = max(n_rows)
            max_rows_history = self.holdings[n_rows.index(max_rows)].history.iloc[:, 0:0]

            for index, holding in enumerate(self.holdings):
                if n_rows[index] < max_rows:
                    holding.history = max_rows_history.join(holding.history, how='left')
                    holding.history = holding.history.interpolate(method='nearest', limit_direction='both')
                    holding.compute_value()
            
    def rebalance(self,rebalance_dates):
        
        # We rebalance on the dates specified. If a rebalancing date is prior a funds 
        # inception date then the fund is ignore in the rebalancing.
        
        # It is assumed that fees are paid for with an external account and that the fees
        # are accrued daily.
        
        holdings = self.holdings
        
        matching_rows = holdings[0].history.index.get_indexer(rebalance_dates,method='nearest')
        
        for row in matching_rows:
            
            # Compute the current allocation
            current_allocation = list()
            for i in range(len(holdings)):
                cum_value = holdings[i].history.iloc[row].loc['cum_value'];
                if cum_value == "nan":
                    # If the deposits pre-date the fund, we use the deposit value in 
                    # the allocation computation.
                    cum_value = holdings[i].history.iloc[row].loc['cum_deposits'];
                current_allocation.append(cum_value)
                
            # Compute target allocations based on asset weightings
            total_value = sum(current_allocation)
            target_allocation = [fraction*total_value for fraction in self.allocation]
            
            # Update units, cum_units and cum_value in light of the new breakdown
            # These are really properties of the holding...
            cum_value_idx = holdings[i].history.columns.get_loc("cum_value")
            deposit_idx = holdings[i].history.columns.get_loc("deposits")
            
            for i in range(len(holdings)):
                holdings[i].history.iloc[row,deposit_idx] = holdings[i].history.iloc[row,deposit_idx] \
                                                                   + (target_allocation[i] - holdings[i].history.iloc[row,cum_value_idx])
                holdings[i].compute_value()
            
            self.holdings = holdings
            return self

Let's check we throw errors in the right conditions...

A method to test that everything is working would be to create a portfolio that consists of only the same funds and compare the cumulate value to a holding.

In [68]:
fixed_allocation_portfolio = FixedAllocationPortfolio(['Vanguard LifeStrategy 100%','Vanguard LifeStrategy 100%'], \
                                                      ['0P0000TKZO.L','0P0000TKZO.L'], \
                                                      [0.22, 0.22],\
                                                      [0.5,0.5], \
                                                      create_monthly_deposits('01/01/2018','01/12/2022',1000))

We don't need to rebalance to check the cumulative value as the funds are the same.

We define a common interface to convert a selection of holdings into a single returns object. We can use the returns object to commonise plotting and visualisation of results. One thing we need is a method to determine whether we have a single holding or a fixed allocation portfolio.

In [69]:
#| export
class Returns:
    def __init__(self,
                 name:    str ,   # Description of the returns - typically used as title
                 holdings: list):  # List of holdings
    
            self.name = name
            self.history = holdings[0].history[["deposits","cum_value","fees"]]
            for i in range(1,len(holdings)):
                self.history += holdings[i].history[["deposits","cum_value","fees"]]

Let's add methods to our `Holding` and `FixedAllocationPortfolio` classes to generate a `Returns` object.

In [70]:
#| export
@patch
def to_returns(self:Holding):
    return Returns(self.fund,[self])

Can we generate our returns for our single holding?

In [71]:
single_holding_returns= single_holding.to_returns()
single_holding_returns.name

'Vanguard LifeStrategy 100%'

In [72]:
single_holding_returns.history.head()

,deposits,cum_value,fees
Date,,,
2018-01-31,1000.0,1000.000000,0.006021
2018-02-01,0.0,998.294418,0.006011
2018-02-02,0.0,988.381070,0.005951
2018-02-05,0.0,968.331418,0.005830
2018-02-06,0.0,964.741112,0.005809


Now, let's do something similar fo the `FixedAllocationPortfolio` class.

In [73]:
#| export
@patch
def to_returns(self:FixedAllocationPortfolio):
    
    name = f"{100*self.allocation[0]}% {self.fund[0]}"
    for i in range(1,len(self.holdings)):
        name += f", {100*self.allocation[i]}% {self.fund[i]}"

    return Returns(name,self.holdings)

Similarly, does it work?

In [74]:
fixed_allocation_portfolio_returns = fixed_allocation_portfolio.to_returns()
fixed_allocation_portfolio_returns.name

'50.0% Vanguard LifeStrategy 100%, 50.0% Vanguard LifeStrategy 100%'

In [75]:
fixed_allocation_portfolio_returns.history.head()

,deposits,cum_value,fees
Date,,,
2018-01-31,1000.0,1000.000000,0.006021
2018-02-01,0.0,998.294418,0.006011
2018-02-02,0.0,988.381070,0.005951
2018-02-05,0.0,968.331418,0.005830
2018-02-06,0.0,964.741112,0.005809


One of the most basic things we might want to do is determine our proft....

In [76]:
#| export
@patch
def profit(self:Returns):
    
    if self.history.iloc[-1].isnull().sum() == 3:
        return self.history['cum_value'][-2]-sum(self.history['deposits'][:-1])-sum(self.history['fees'][:-1])
    
    return self.history['cum_value'][-1]-sum(self.history['deposits'])-sum(self.history['fees'])

and let's see if this works for our single holding and our portfolio. Let's start with the the single holding.

In [77]:
single_holding_returns.profit()

12869.657880599709

Here, we can check our `FixedAllocationPortfolio` and `Holding` classes give the same result for identical fund allocations.

In theory, rebalancing here should make no difference. Let's create a helper function to create a monthly rebalancing schedule...

In [78]:
#| export
create_monthly_rebalance_dates =  lambda start, end: pd.bdate_range(start=to_datetime(start),end=to_datetime(end),freq='BM')

In [79]:
create_monthly_rebalance_dates('01/01/2018','01/12/2022')

DatetimeIndex(['2018-01-31', '2018-02-28', '2018-03-30', '2018-04-30',
               '2018-05-31', '2018-06-29', '2018-07-31', '2018-08-31',
               '2018-09-28', '2018-10-31', '2018-11-30', '2018-12-31',
               '2019-01-31', '2019-02-28', '2019-03-29', '2019-04-30',
               '2019-05-31', '2019-06-28', '2019-07-31', '2019-08-30',
               '2019-09-30', '2019-10-31', '2019-11-29', '2019-12-31',
               '2020-01-31', '2020-02-28', '2020-03-31', '2020-04-30',
               '2020-05-29', '2020-06-30', '2020-07-31', '2020-08-31',
               '2020-09-30', '2020-10-30', '2020-11-30', '2020-12-31',
               '2021-01-29', '2021-02-26', '2021-03-31', '2021-04-30',
               '2021-05-31', '2021-06-30', '2021-07-30', '2021-08-31',
               '2021-09-30', '2021-10-29', '2021-11-30', '2021-12-31',
               '2022-01-31', '2022-02-28', '2022-03-31', '2022-04-29',
               '2022-05-31', '2022-06-30', '2022-07-29', '2022-08-31',
      

... and check our profit does not change.

In [80]:
fixed_allocation_portfolio_rebalanced = fixed_allocation_portfolio.rebalance(create_monthly_rebalance_dates('01/01/2018','01/12/2022'))
assert(single_holding_returns.profit() == fixed_allocation_portfolio_rebalanced.to_returns().profit())

This was quite an easy test as the funds were identical. Let see if we can use a single `FixedAllocationPortfolio` to give the same profit as two holdings where we have done a manual rebalance. Let's check how the rebalance and deposit dates align so tha we have an identical computation.

In [81]:
create_monthly_rebalance_dates('01/01/2021','29/01/2021')

DatetimeIndex(['2021-01-29'], dtype='datetime64[ns]', freq='BM')

In [82]:
create_monthly_deposits('30/11/2020','01/02/2021',0.8*1000)

2020-11-30    800.0
2020-12-31    800.0
2021-01-29    800.0
Freq: BM, Name: deposits, dtype: float64

Now, we create our desposits...

In [83]:
holding_one_deposits = create_monthly_deposits('01/01/2018','31/12/2020',0.8*1000)

but also note the length of this array as we are going to do our final rebalance on the day that would be the next monthl deposit.

In [84]:
rebalance_row = holding_one_deposits.shape[0]

We create our holdings with the deposit time series...

In [85]:
holding_one = Holding("Scottish Mortgage Ord","GB00BLDYK618",0.1,holding_one_deposits)

and also note the row where our next deposit it will be. The reader should be aware that holdings track from the date of the first deposit to the present day.

In [86]:
row = holding_one.history.index.get_indexer([to_datetime('29/01/2021')], method='nearest')[0]
row

758

In [87]:
holding_one.history.index[row]

Timestamp('2021-01-29 00:00:00')

In [88]:
holding_one_value  = holding_one.history.cum_value[row]
holding_one_value

60231.05760583867

In [89]:
holding_two = Holding("Fundsmith Equity I Acc","GB00B41YBW71",0.1,create_monthly_deposits('01/01/2018','31/12/2020',0.2*1000))
holding_two_value  = holding_two.history.cum_value[row]
holding_two_value

8399.1920909121

In [90]:
total_value = holding_one_value + holding_two_value
total_value

68630.24969675078

Now, let's make create two holdings for our whole duration and update the deposits to do the manual rebalance...

In [91]:
holding_one_deposits = create_monthly_deposits('01/01/2018','31/12/2022',0.8*1000)
holding_one_deposits[rebalance_row] = holding_one_deposits[rebalance_row]+(0.8*total_value-holding_one_value)
holding_one = Holding("Scottish Mortgage Ord","GB00BLDYK618",0.1,holding_one_deposits)
holding_two_deposits = create_monthly_deposits('01/01/2018','31/12/2022',0.2*1000)
holding_two_deposits[rebalance_row] = holding_two_deposits[rebalance_row]+(0.2*total_value-holding_two_value)
holding_two = Holding("Fundsmith Equity I Acc","GB00B41YBW71",0.1,holding_two_deposits)

and do the same with our `FixedAllocationPortfolio`.

In [92]:
multi_holding_portfolio = FixedAllocationPortfolio(["Scottish Mortgage Ord","Fundsmith Equity I Acc"],
                                                   ["GB00BLDYK618","GB00B41YBW71"],
                                                   [0.1, 0.1],
                                                   [0.8, 0.2],
                                                   create_monthly_deposits('01/01/2018','31/12/2022',1000))
rebalanced_multi_holding_portfolio = multi_holding_portfolio.rebalance(create_monthly_rebalance_dates('01/01/2021','29/01/2021'))


In [93]:
assert(holding_one.to_returns().profit() + holding_two.to_returns().profit() - rebalanced_multi_holding_portfolio.to_returns().profit() < 0.01)

In [122]:
monthly_deposits = create_monthly_deposits("01/01/2018",date.today().strftime("%d/%m/%Y"),20000/12)
active_growth = FixedAllocationPortfolio(["Jupiter UK Special Situations Acc",
                                    "Ninety One UK Alpha Acc",
                                    "Scottish Mortgage Ord",
                                    "Fidelity Global Dividend W Acc",
                                    "Fundsmith Equity",
                                    "F&C Investment Trust",
                                    "Vanguard U.S. Equity",
                                    "JPMorgan Emerging Markets",
                                    "Capital Gearing Ord",
                                    "LF Ruffer Diversified",
                                    "abrdn Private Equity Opportunities"],
                                               ["0P0000K9EK.L",
                                                "0P00012K82.L",
                                                "SMT.L",
                                                "0P0000WUT3.L",
                                                "GB00B4Q5X527.L",
                                               "FCIT.L",
                                               "0P0000KSPA.L",
                                               "JMG.L",
                                               "CGT.L",
                                               "0P0001MKQK.L",
                                               "APEO.L"],
                                               [0.76,0.74,0.84,0.91,0.94,0.83,0.1,0.86,0.66,1.12,6.25],
                                               [0.1 , 0.1,0.12, 0.1,0.13, 0.1,0.1,0.05,0.10,0.05,0.05],
                                               monthly_deposits)

C:\Users\Ryan\AppData\Local\Temp\ipykernel_29804\955690272.py:25: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  if not deposits.empty and deposits.index[0].date() == self.history.index[0]:
C:\Users\Ryan\AppData\Local\Temp\ipykernel_29804\955690272.py:31: UserWarning: Deposits predate initial date of 2021-09-01 00:00:00 where prices are available for LF Ruffer Diversified. Accumulated deposit of 3666.666666666668 deposited on this date.
  warnings.warn(warn_msg)


In [128]:
active_growth.holdings[0].history.head()

,Open,High,Low,Close,Volume,deposits,cum_deposits,units,cum_units,cum_value,fees
Date,,,,,,,,,,,
2018-01-31,240.399994,240.399994,240.399994,240.399994,0.0,166.666667,166.666667,0.693289,0.693289,166.666667,0.003457
2018-02-01,238.929993,238.929993,238.929993,238.929993,0.0,0.000000,166.666667,0.000000,0.693289,165.647531,0.003436
2018-02-02,236.949997,236.949997,236.949997,236.949997,0.0,0.000000,166.666667,0.000000,0.693289,164.274822,0.003408
2018-02-05,233.899994,233.899994,233.899994,233.899994,0.0,0.000000,166.666667,0.000000,0.693289,162.160288,0.003364
2018-02-06,229.160004,229.160004,229.160004,229.160004,0.0,0.000000,166.666667,0.000000,0.693289,158.874105,0.003296


In [114]:
active_growth_rebalanced = create_monthly_rebalance_dates("01/01/2018",date.today().strftime("%d/%m/%Y"))
active_growth_rebalanced = active_growth.rebalance(rebalance_dates)

In [119]:
active_growth_rebalanced.holdings[4].history.head()b

,Open,High,Low,Close,Volume,deposits,cum_deposits,units,cum_units,cum_value,fees
Date,,,,,,,,,,,
2018-01-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-02-01,NaN,NaN,NaN,NaN,NaN,216.666667,216.666667,NaN,NaN,NaN,NaN
2018-02-02,NaN,NaN,NaN,NaN,NaN,216.666667,433.333333,NaN,NaN,NaN,NaN
2018-02-05,NaN,NaN,NaN,NaN,NaN,216.666667,650.000000,NaN,NaN,NaN,NaN
2018-02-06,NaN,NaN,NaN,NaN,NaN,216.666667,866.666667,NaN,NaN,NaN,NaN


In [120]:
rebalance_dates

DatetimeIndex(['2018-01-31', '2018-02-28', '2018-03-30', '2018-04-30',
               '2018-05-31', '2018-06-29', '2018-07-31', '2018-08-31',
               '2018-09-28', '2018-10-31', '2018-11-30', '2018-12-31',
               '2019-01-31', '2019-02-28', '2019-03-29', '2019-04-30',
               '2019-05-31', '2019-06-28', '2019-07-31', '2019-08-30',
               '2019-09-30', '2019-10-31', '2019-11-29', '2019-12-31',
               '2020-01-31', '2020-02-28', '2020-03-31', '2020-04-30',
               '2020-05-29', '2020-06-30', '2020-07-31', '2020-08-31',
               '2020-09-30', '2020-10-30', '2020-11-30', '2020-12-31',
               '2021-01-29', '2021-02-26', '2021-03-31', '2021-04-30',
               '2021-05-31', '2021-06-30', '2021-07-30', '2021-08-31',
               '2021-09-30', '2021-10-29', '2021-11-30', '2021-12-31',
               '2022-01-31', '2022-02-28', '2022-03-31', '2022-04-29',
               '2022-05-31', '2022-06-30', '2022-07-29', '2022-08-31',
      

We might be interested in plotting the value of a holding over time. Let's patch in a method for that. `yfinfance` and `matplotlib` can be used if we want to just the price of a stock over time.

In [97]:
# def plot(holding):
#     'Plot value of the holding'
#     plt.plot(holding.history['cum_value'])
#     plt.xticks(rotation = 90)
#     plt.title(holding.fund)
#     return plt

We use the `MonthLocator` to set the xtick positions.

In [98]:
# from matplotlib.dates import MonthLocator
# plt = plot(single_holding)
# ax = plt.gca()
# ax.xaxis.set_major_locator(MonthLocator([1,7]))

In [99]:
#| hide
import nbdev; nbdev.nbdev_export()